# Pénzügyi adatelemzés 
### Második házi feladat
### Garbage in - garbage out

A házi feladatot itt a notebook-ban kell megoldani
- legfeljebb 4 fős csapatokban és
- ipynb (Jupyter Notebook) formátumban a Moodle-re feltölteni <span style="color:red">**2025-11-09 vasárnap este 23:55-ig**</span> csapatonként EGY példányban (tehát NE töltse fel mindenki).
- A csapatokra vonatkozólag nincs más megkötés, mint, hogy legfeljebb 4 fősek lehetnek, lehet dolgozni akár párban, akár egyedül is, nem szükséges a csapattagoknak ugyanabba a szemináriumi csoportba járni, és eltérhetnek az első házi feladat csapataitól. 
- A csapattagok nevét, Neptun-kódját és "uni-corvinus"-os e-mail címét írjátok be a notebook második cellájába. Minden feltöltött file-ra az a hallgató fog pontot kapni, akinek az adatai itt hibátlanul szerepelnek. Ha ugyanaz a csapat több file-t is feltölt, akkor pontot vonunk le. 

Minden kérdés után van a szöveges válasznak vagy kódnak egy üres cella, de természetesen lehet újabb cellákat is beszúrni és további függvényeket írni.

Nem feltétlenül elegendő az előre megírt függvényeket kiegészíteni, a kódnak minden, a feladatben kért output-ot el kell készítenie és a cellákra sorban kattintva hibátlanul lefutnia. A feladat értékelésénél nem fogunk semmilyen további kódot hozzáírni, ha valaki elfelejt meghívni egy függvényt, akkor pontot veszíthet. Mivel a notebook egyetlen session, emlékszik minden változóra, akkor is, ha az azt létrehozó kódot később kitöröljük, így ellenőrzésként célszerű a végső verzió tesztelése előtt újraindítani a kernel-t vagy bezárni a file-t újraindítani a szervert, és úgy kipróbálni.

Az elméleti kérdésekre adott válaszok legyenek rövidek, pontosak, lényegre törőek, a szükséges mennyiségű, de nem több indoklással. Aki válaszként mindent is leír, (ahogy a chatGPT szokta), azt negatívan értékeljük. Az ábrákat nem kell "szép"-re formázni, megfelelnek az alapértelmezett beállítások. A kódok a kért feladatokat oldják meg, ne valami mást vagy általánosabbat, a számítások egyes lépéseit szükség esetén egy-két szavas kommentekkel el lehet látni.

AI indokolt esetben és módon használható, de meg kell jelölni, hogy hol, mire és miért használtátok. Ha Python kód generálásra használtok generatív AI-t, akkor másoljátok ki a promt-ot (az AI-nak adott utasítást) egy külön cellába.

A szimulációk paramétereit a beadott file-ban úgy állítsátok be, hogy a kódok futása gyors (legfeljebb néhány másodperc) legyen, de a szöveges válaszokban összefoglalhatjátok a szimulációk eredményeit nagyobb mintaelemszámra is. Sokáig futó kódhoz célszerű a `tqdm` package `tqdm` függvényét használni, ami egy szépen formázott progressbart készít egy `for` ciklushoz. 

**Csapattagok (név, Neptun-kód, e-mail cím):**

A feladatok célja azt illusztrálni, hogyan hat a regressziós modellre ha "ész nélkül" minden változót beleteszünk, ami rendelkezésre áll.  


In [2]:
# A cellákban külön importokat ne tegyünk, egy helyen töltsük be az összes felhasznált package-t. 
# Az összes feladat megoldható ezekkel (de persze lehet bővíteni a listát):

from matplotlib import pyplot as plt
import numpy as np
import statsmodels.api as sm
import seaborn as sns
from tqdm import tqdm

#### 1. Feladat

Legyen $k$, $l$ és $m$ pozitív egész számok, $x_1, x_2, \dots, x_k$ és $\varepsilon$ független standard normális eloszlású valószínűségi változók, $\sigma\in\mathbb{R}$ egy pozitív szám és tekintsük az
\begin{equation}
Y = \sum_{i = 1}^k \frac{(k-i+1)}{k} x_i + \sigma\varepsilon
\tag{E}
\end{equation}
regressziós egyenletet. Az adatgeneráló folyamat tehát tartalmaz $k$ darab magyarázó változót egyre csökkenő bétákkal tehát, például $k=5$-re $$Y = x_1 + \frac{4}{5}x_2 + \frac{3}{5}x_3 + \frac{2}{5}x_4 + \frac{1}{5}x_5 + \sigma\varepsilon$$.

Írjunk egy függvényt, ami adott $m$, $k$, $l$ és $\sigma$ függvényében generál egy olyan adattáblát, amely $m$ darab megfigyelést ($m$ sort) tartalmaz 

- a fenti $k$ darab független standard normális eloszlású magyarázó változóból,
- $l$ darab ún. zajváltozóból (melyek mind egymástól, mind a többi $x_i$-től független standard normális eloszlású változók),
- és az $(E)$ egyenlet szerinti célváltozóból, melyeknek tehát semmi közük a zajváltozókhoz.

Az adattáblát a későbbi kényelmesebb használat érdekében úgy is létrehozhatjuk, hogy csak a magyarázó változókat és a zajváltozókat (tehát csak $x$-eket) tartalmazza, a célváltozót pedig külön egy `tuple` második elemeként adja vissza a függvény.  


In [6]:
def generate_data(m, k, l, sigma=1):
    x=np.random.normal(size=(m,k))
    z=np.random.normal(size=(m,l))
    e=np.random.normal(size=m)
    beta=np.array([(k-i+1)/k for i in range(1, k+1)])
    y=x@beta+sigma*e
    X=np.hstack((x,z))
    return X, y

#print(generate_data((100, 2, 2,))    

(array([[ 0.46066043, -2.50006123, -1.43461407,  0.86016569],
       [-0.61353327, -1.04567133,  1.56221915, -0.51044395],
       [ 1.09358048,  0.61976025,  0.39160608, -1.14263214],
       [ 0.66973875,  0.53073713,  0.26509289, -1.08131388],
       [-1.39968146,  0.7549887 , -0.37405098,  0.70374174],
       [-1.11934004, -0.42474669, -0.34385678,  2.51383365],
       [-2.26090166,  0.15890355,  1.77155872, -0.79710733],
       [-1.39891799,  0.12991262, -0.52457806,  1.45082105],
       [ 0.39313691, -0.31446644,  0.1224405 ,  0.79167394],
       [ 0.77695531, -0.63641991,  0.10030517, -0.68949662],
       [-0.36691143,  1.32032096,  0.65268729,  0.19322931],
       [-0.25331078, -0.68606784,  0.34344382,  0.5360205 ],
       [ 1.40794177,  0.892628  , -0.42408997,  0.22567508],
       [-1.4468152 , -1.41007802, -0.81280695, -0.32910182],
       [-0.27821335,  0.28145897,  1.39112118, -0.72244216],
       [ 2.1935407 ,  1.49039382, -0.18322907,  1.52145947],
       [ 0.01164873,  0

#### 2. Feladat

Írjunk egy függvényt, amely az adatok és egy $\alpha$ signifikanciaszint alapján megbecsüli a regressziós modellt és visszaad egy $k+l$ elemű vektort (listát vagy tuple-t vagy numpy array-t), amely az összes magyarázó változó (a zajváltozókat is beleértve) $t$-tesztjének $p$-értékei alapján egy logikai értékekből álló vektort ad vissza, annak megfelelően, hogy az adott változó $t$-tesztje a megadott $\alpha$ szinten szignifikáns-e.

Elméleti kérdés: egy "tökéletes világban" milyen eredményeket (milyen True/False értékeket) kéne ennek a függvénynek visszaadni?

In [ ]:
def beta_sign_test(data, alpha = 0.95):
    return

#### 3. Feladat

Legyen az egyszerűség kedvéért $\alpha = 95\%$, $\sigma = 1$ és $k = 5$. Futtassunk szimulációt a 2. feladatban megírt függvény használatával az $l$ (a zajváltozók száma) és az $m$ (megfigyelések száma) különböző kombinációira, és próbáljuk meg szimulációval megállapítani, hogy adott számú zajváltozó esetén hány megfigyelés kell ahhoz, hogy "nagy valószínűséggel jó eredményt" kapjunk a feladatból, tehát mondjuk 90%-os valószínűséggel a modell válassza szét, a zajváltozókat és a releváns változókat. 

Megjegyzés: Itt nem várunk feltétlenül tökéletesen egzakt megoldást, de mondjatok valami értelmeset arról, hogy mennyi adat biztos, hogy nem elég a megbízható eredményhez, és mennyi adat lesz már nagyon nagy valószínűséggel elég. A főbb megállapításokat foglaljátok össze szövegesen, illetve további opcionális (nem kötelező) feladatként készítsetek három  táblázatot, amelyben különböző $l$ és $m$ kombinációkra szimuláció alapján annak a valószínűsége van, hogy

- a modell legalább 1 releváns magyarázó változót megtalál,
- a modell nem talál szignifikánsnak egyetlen zajváltozót sem,
- a modell felismeri az adatgeneráló folyamatot, azaz szét tudja választani a releváns magyarázó változókat a zajtól.

Azt itt nem kell értékelni, hogy mekkora hibával találja meg a modell bétákat, csak azt, hogy melyiket tekinti nullának illetve nem nullának a $t$-teszt). A táblázatot olyan $l$ és $m$ kombinációkból rakjátok össze, hogy abban a releváns eredmények látszódjanak, tehát nemtriviális átmenetek nem csak 0-k és 1-esek.

In [ ]:
# SZIMULÁCIÓ

<div style="color:blue">
Konklúziók
</div>